In [1]:
!pip install yfinance
!pip install yahooquery
!pip install tvdatafeed
!pip install tensorflow
from tvDatafeed import TvDatafeed, Interval
from yahooquery import Screener
import yfinance as yf   
import pandas as pd
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Flatten
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import random 
from tensorflow.keras.models import load_model
import os
import shutil
from datetime import datetime
import glob

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.4 MB/s 
     |████████████████████████████████| 6.4 MB 43.6 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.28.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |██████████

In [2]:
def work_with_dir():
  if os.path.exists("/content/data/"):
    shutil.rmtree("/content/data/", ignore_errors=True)
    print("Data Folder Removed")
    os.mkdir("/content/data/")
  if not os.path.exists("/content/data/"):
    os.mkdir("/content/data/")
  if not os.path.exists("/content/extracted/"):
    os.mkdir("/content/extracted/")
def get_crypto_syms():
   # 'all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in',
   screens = [
       'all_cryptocurrencies_us', 'all_cryptocurrencies_au', 'all_cryptocurrencies_ca', 'all_cryptocurrencies_eu', 'all_cryptocurrencies_gb', 'all_cryptocurrencies_in', ]
   s = Screener()
   symbols = []
   for i in screens:
      data = s.get_screeners(i, count=250)
      dicts = data[i]['quotes']
      syms = [d['symbol'] for d in dicts]
      for sym in syms:
        symbols.append(sym)
   # print(len(symbols))
   # pieces = 15
   # new_arrays = np.array_split(symbols, pieces)
   return symbols

def download_data(symbols,periodd,intervall):
  indexx = 100
  work_with_dir()
  for symbol in symbols:
    if ((symbols.index(symbol)+1) % 100 == 0):
      print(f" -- {indexx}",end="")
      indexx = indexx + 100
    try:
        data = yf.download(symbol, period=periodd,interval=intervall, progress=False,show_errors=False)
        if data.empty:
           pass
        else:
             data.to_csv(f"/content/data/{symbol}.csv")         
    except:
       print("Error!")
  print(" ")

def extract_data(pieces,min_range,max_range,how_many_future_candles,how_many_past_candles,each_row_past,interval_for_that):
  print(f"Files In Data : {len(os.listdir('/content/data/'))}")
  pd.options.mode.chained_assignment = None
  files = os.listdir("/content/data/")
  new_files = np.array_split(files, pieces)
  print("Processing File:")
  now = datetime.now().strftime("%H%M%S")
  os.mkdir(f"/content/extracted/{now}/")
  index = 0 
  for files in new_files:
     each_file_proc(files,now,index,min_range,max_range,how_many_future_candles,how_many_past_candles,each_row_past,interval_for_that)
     index = index + len(files)
  print(" ")
  return now

def make_df(now):
   path = f'/content/extracted/{now}/*.npy'
   files = glob.glob(path)
   data = []
   for adr in files:
     data.append(np.load(adr))
   data = np.concatenate(data)
   print(data.shape)
   np.save(f'/content/extracted/{now}/data',data)
   return data
def to_par(now,howmanyfiles): 
    files = os.listdir(f"/content/extracted/{now}/")
    addresses = []
    for file in files:
      addresses.append(f"/content/extracted/{now}/{file}")
    new_adr = np.array_split(addresses,howmanyfiles)
    for adrs in new_adr:
      datas = []
      for adr in adrs:
        datas.append(np.load(adr))
      datas = np.concatenate(datas)
      rnow = datetime.now().strftime("%H%M%S%f")
      np.save(f"/content/extracted/{now}/part_{rnow}",datas)   
    for adr in addresses:
      os.remove(adr)
def scaler(row,min_range,max_range):
    scaler = MinMaxScaler(feature_range=(min_range, max_range))
    row = scaler.fit_transform(row)
    return row

def each_file_proc(files,now,index,min_range,max_range,how_many_future_candles,how_many_past_candles,each_row_past,interval_for_that):
     data = []
     unattached_dfs = []
     files = list(files)
     for file in files:
        #print(f"{files.index(file)+1+index}",end=" ")
        if (files.index(file)+index+1) % 10 == 0:
           print(f"{files.index(file)+1+index}",end=" ")
           if (files.index(file)+index+1) % 400 == 0:
             print(" ")

        address = f"/content/data/{file}"
        #try:
        unattached_dfs.append(process(pd.read_csv(address),min_range,max_range,how_many_future_candles,how_many_past_candles,each_row_past,interval_for_that))
        #except:
        #  pass
     nud = []
     for i in unattached_dfs:
       if i.size != 0:
         nud.append(i)
     data = np.concatenate(nud)
     right_now = datetime.now().strftime("%H%M%S%f")
     np.save(f"/content/extracted/{now}/{right_now}",data)  
def process(data,min_range,max_range,how_many_future_candles,how_many_past_candles,each_row_past,interval_for_that):
    data = data.dropna()
    row = []
    if len(data.columns) == 7:
      data = data.iloc[:, 1:]
    data = np.array(data)
    for i in range(each_row_past, data.shape[0]-how_many_future_candles):
        row_del = []          
        odel = []
        hdel = []
        ldel = []
        cdel = []
        for ix in range(each_row_past, 1,interval_for_that):
            odel.append([data[i-ix][0] - data[i-ix+1][0]])
            hdel.append([data[i-ix][1] - data[i-ix+1][1]])
            ldel.append([data[i-ix][2] - data[i-ix+1][2]])
            cdel.append([data[i-ix][3] - data[i-ix+1][3]])

        
        
        row_del = np.array([odel,hdel,ldel,cdel])
        
        scaled_row_del = []
        for m in row_del:
          m = np.array(m)
          sc = scaler(m,min_range,max_range)
          scaled_row_del.append(sc)
        sugg = None
        if data[i+5][0] > data[i-1][3]:
          sugg= 1
        else:
          sugg = 0
        
        scaled_row_del[0] = np.append(scaled_row_del[0],sugg)
        scaled_row_del[1] = np.append(scaled_row_del[1],sugg)
        scaled_row_del[2] = np.append(scaled_row_del[2],sugg)
        scaled_row_del[3] = np.append(scaled_row_del[3],sugg)

        row.append(np.array(scaled_row_del).reshape(4,-1))
    return np.array(row)
def start(min_range,max_range,how_many_future_candles,how_many_past_candles,each_row_past,interval_for_that,into_how_many_csv,csv_to_parquet):
     folder_name = extract_data(into_how_many_csv,min_range,max_range,how_many_future_candles,how_many_past_candles,each_row_past,interval_for_that)
     to_par(folder_name,csv_to_parquet)
     data = make_df(folder_name)
     return data

In [3]:
symbols = get_crypto_syms()
print(f"Symbols : {len(symbols)}")
download_data(symbols,"max","1wk")

Symbols : 1500
 -- 100 -- 200 -- 300 -- 400 -- 500 -- 600 -- 700 -- 800 -- 900 -- 1000 -- 1100 -- 1200 -- 1300 -- 1400 -- 1500 


In [4]:

min_range,max_range = -1,1
how_many_future_candles = 7
how_many_past_candles = 7
each_row_past,interval_for_that = 90,-1

into_how_many_csv = 1
csv_to_parquet = 1
data = start(min_range,max_range,how_many_future_candles,how_many_past_candles,each_row_past,interval_for_that,into_how_many_csv,csv_to_parquet)
#print(yTrain.value_counts())
#print(yTest.value_counts())

Files In Data : 1500
Processing File:
10 20 30 40 50 60 70 80 90 100 110 120 130 140 150 160 170 180 190 200 210 220 230 240 250 260 270 280 290 300 310 320 330 340 350 360 370 380 390 400  
410 420 430 440 450 460 470 480 490 500 510 520 530 540 550 560 570 580 590 600 610 620 630 640 650 660 670 680 690 700 710 720 730 740 750 760 770 780 790 800  
810 820 830 840 850 860 870 880 890 900 910 920 930 940 950 960 970 980 990 1000 1010 1020 1030 1040 1050 1060 1070 1080 1090 1100 1110 1120 1130 1140 1150 1160 1170 1180 1190 1200  
1210 1220 1230 1240 1250 1260 1270 1280 1290 1300 1310 1320 1330 1340 1350 1360 1370 1380 1390 1400 1410 1420 1430 1440 1450 1460 1470 1480 1490 1500  
(139778, 4, 90)


In [5]:
percent_of_train = 90
numb = data.shape[0] * percent_of_train / 100
numb = round(numb)
print(f"Number of Train : {numb}")
print(f"Number of Test  : {data.shape[0]-numb}")
xTrain = data[:numb]
xTest = data[numb:]

Number of Train : 125800
Number of Test  : 13978


In [6]:
yTrain = []
counts_of_b = 0
counts_of_s = 0
for i in xTrain:
  if i[0][-1] == 1:
    yTrain.append([1,0])
    counts_of_b += 1
  else:
    yTrain.append([0,1])
    counts_of_s += 1
yTest = []
for i in xTest:
  if i[0][-1] == 1:
    yTest.append([1,0])
    counts_of_b += 1

  else:
    yTest.append([0,1])
    counts_of_s += 1

yTrain = np.array(yTrain)
yTest = np.array(yTest)

xTrain = np.delete(xTrain,89,axis=2)
print(xTrain.shape)

xTest = np.delete(xTest,89,axis=2)
print(xTest.shape)

print(f"Buys : {counts_of_b}")
print(f"Sell : {counts_of_s}")

(125800, 4, 89)
(13978, 4, 89)
Buys : 65512
Sell : 74266


In [44]:
model = Sequential()

model.add(Dense(1024, activation='relu', input_shape=(4,89)))
model.add(Dense(1024, activation='relu'))
model.add(Dense(892, activation='relu'))
model.add(Dense(892, activation='relu'))
model.add(Dense(750, activation='relu'))
model.add(Flatten())
model.add(Dense(750, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(2, activation='sigmoid'))

opt = tf.keras.optimizers.Adamax()

model.compile(optimizer=opt, loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 4, 1024)           92160     
                                                                 
 dense_9 (Dense)             (None, 4, 1024)           1049600   
                                                                 
 dense_10 (Dense)            (None, 4, 892)            914300    
                                                                 
 dense_11 (Dense)            (None, 4, 892)            796556    
                                                                 
 dense_12 (Dense)            (None, 4, 750)            669750    
                                                                 
 flatten_1 (Flatten)         (None, 3000)              0         
                                                                 
 dense_13 (Dense)            (None, 750)              

In [46]:
model.fit(xTrain,yTrain,epochs=15,batch_size=1024,validation_data=(xTest,yTest))
acr = str(round(model.evaluate(xTest,yTest)[1],4)).replace("0.","")
model.save(f"{acr}.h5")

Epoch 1/15
123/123 [==============================] - 203s 2s/step - loss: 0.0255 - accuracy: 0.9880 - val_loss: 0.1432 - val_accuracy: 0.9757
Epoch 2/15
123/123 [==============================] - 198s 2s/step - loss: 0.0229 - accuracy: 0.9888 - val_loss: 0.1588 - val_accuracy: 0.9758
Epoch 3/15
123/123 [==============================] - 199s 2s/step - loss: 0.0222 - accuracy: 0.9892 - val_loss: 0.1752 - val_accuracy: 0.9756
Epoch 4/15
123/123 [==============================] - 198s 2s/step - loss: 0.0219 - accuracy: 0.9893 - val_loss: 0.1868 - val_accuracy: 0.9756
Epoch 5/15
123/123 [==============================] - 200s 2s/step - loss: 0.0217 - accuracy: 0.9894 - val_loss: 0.1917 - val_accuracy: 0.9759
Epoch 6/15
123/123 [==============================] - 199s 2s/step - loss: 0.0216 - accuracy: 0.9894 - val_loss: 0.1965 - val_accuracy: 0.9757
Epoch 7/15
123/123 [==============================] - 199s 2s/step - loss: 0.0215 - accuracy: 0.9896 - val_loss: 0.1961 - val_accuracy: 0.9760

In [49]:
model.predict(np.array(xTest[450]).reshape(1,4,89))

array([[1.5185869e-07, 1.0000000e+00]], dtype=float32)

In [50]:
print(yTest[450])

[0 1]


In [39]:
def process_for_prediction(data,index,min_range,max_range,each_row_past,interval_for_that):
        i = index
        if "symbol" in data.columns:
              data.drop("symbol",axis=1,inplace=True) 
        if "datetime" in data.columns:
              data.drop("datetime",axis=1,inplace=True)
        if "Adj Close" in data.columns:
              data.drop("Adj Close",axis=1,inplace=True)

        data = np.array(data)

        odel = []
        hdel = []
        ldel = []
        cdel = []
        for ix in range(each_row_past, 1,interval_for_that):
            odel.append([data[i-ix][0] - data[i-ix+1][0]])
            hdel.append([data[i-ix][1] - data[i-ix+1][1]])
            ldel.append([data[i-ix][2] - data[i-ix+1][2]])
            cdel.append([data[i-ix][3] - data[i-ix+1][3]])

        row_del = np.array([odel,hdel,ldel,cdel])
        scaled_row_del = []
        for m in row_del:
          m = np.array(m)
          sc = scaler(m,min_range,max_range)
          scaled_row_del.append(sc)
        scaled_row_del = np.array(scaled_row_del).reshape(1,4,89)
        return scaled_row_del,0
def make_prediction_for_yf(symbol,period,timeframe,index,min_range,max_range,each_row_past,interval_for_that):
    raw_data, mean_of_previous = process_for_prediction(
        yf.download(symbol, period=period, interval=timeframe), index,min_range,max_range,each_row_past,interval_for_that)
    print(model.predict(raw_data))
    return f"""YF   : {model.predict(np.array(raw_data)).tolist()}
MP   : {mean_of_previous}
"""
def make_prediction_for_tv(symbol,exchange,timeframe,tindex,min_range,max_range,each_row_past,interval_for_that):
    tv = TvDatafeed()
    raw_data, mean_of_previous = process_for_prediction(
       tv.get_hist(symbol=symbol, exchange=exchange, interval=timeframe, n_bars=50000), tindex,min_range,max_range,each_row_past,interval_for_that)
    return f"""TVB  : {model.predict(np.array(raw_data)).tolist()}
MP   : {mean_of_previous}
"""
def make_prediction(ysymbol,period,timeframe,tsymbol,texchange,ttimeframe,tindex,index):
  print(make_prediction_for_yf(ysymbol,period,timeframe,index,min_range,max_range,each_row_past,interval_for_that))
  print(make_prediction_for_tv(tsymbol,texchange,ttimeframe,tindex,min_range,max_range,each_row_past,interval_for_that))

In [40]:
make_prediction("btc-usd","max","1d","btcusd","bitstamp",Interval.in_daily,-1,-1)

[*********************100%***********************]  1 of 1 completed


[[2.5218725e-04 9.9999195e-01]]
YF   : [[0.00025218725204467773, 0.9999919533729553]]
MP   : 0

TVB  : [[0.0001098569919122383, 0.9999977350234985]]
MP   : 0



In [ ]:
yf.download("btc-usd",period="max",interval="1wk")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')